# Текстовый RAG на Gigachat

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

root_path = "/content/drive/MyDrive/Diploma-mag"


In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

from langchain_gigachat import Gigachat
from langchain_gigachat.embeddings import GigaChatEmbeddings


load_dotenv(find_dotenv())

def init_gigachat():
    return Gigachat(credentials="ключ_авторизации", model="GigaChat-Max", verify_ssl_certs=False, temperature=1e-15, timeout=100)


def init_gigachat_embeddings():
    return GigaChatEmbeddings(credentials="ключ_авторизации", scope-"GIGACHAT_API_PER", verify_ssl_certs=False)

In [ ]:
llm = init_gigachat()
embeddings = init_gigachat_embeddings()

In [ ]:
model_id = "gigachat"
text_embedding_model = "gigaEmbeddings"
text_vectorstore_dir = f"./data/text_only_rag/text_only_{model_id}_vectorstore_{text_embedding_model}"

In [ ]:
from langchain_chroma import Chroma
from chromadb.config import Settings

text_vectorstore = Chroma(
    persist_directory=text_vectorstore_dir,
    embedding_function=embeddings,
    collection_name="mm_rag_text_gigaembeddings",
    client_settings=Settings(anonymized_telemetry=False)
)



In [ ]:
import uuid
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)

documents_content = []
documents_page = []

for d in documents:
    documents_content.append(d["text"])
    documents_page.append(d["metadata"]["page_number"])

prepared_text = []
for i, document_content in enumerate(documents_content):
    chunks = text_splitter.split_text(document_content)
    prepared_text += [Document(
        page_content=chunk,
        metadata={
            "page_number": documents_page[i]
        })
    for j, chunk in enumerate(chunks)]

all_chunks = [text.page_content for text in prepared_text]

text_vectorstore.add_documents(prepared_text)

In [ ]:
from langchain_core.messages import HumanMessage
from langchain.prompts import ChatPromptTemplate
from prompts import QA_PROMPT_SYSTEM, QA_PROMPT_USER

def run_pipeline(question, text_vectorstore, llm, debug=False):
    text_content = text_vectorstore.similarity_search(question, k=4)
    context = "\n\n".join([t.page_content for t in text_content])

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", QA_PROMPT_SYSTEM),
            ("human", QA_PROMPT_USER)
        ]
    )
    chain = prompt | llm

    #print(prompt.format(question=question, context=context))

    if debug:
        return chain.invoke({"question": question, "context": context}).content, context
    return  chain.invoke({"question": question, "context": context}).content

In [ ]:
question = "Как выделить прямоугольную область на изображении в Adobe Photoshop?"
llm = init_gigachat()
run_pipeline(question, text_vectorstore, llm)

# Текстовый RAG на LLaMa-3.2-1B

In [ ]:
!pip install -q -U transformers bitsandbytes accelerate langchain torchfrom transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import HuggingFacePipeline
import torch

model_id = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=root_path + "/models")

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    cache_dir=root_path + "/models"
)

# Создание пайплайна
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=200,
    temperature=0.7,
    repetition_penalty=1.1
)

In [ ]:
question = "Как выделить прямоугольную область на изображении в Adobe Photoshop?"
llm = HuggingFacePipeline(pipeline=pipe)
run_pipeline(question, text_vectorstore, llm)